## Import libraries

In [5]:
import ee
import geemap
import apifunction
geemap.update_package()

ModuleNotFoundError: No module named 'apifunction'

## Create an interactive map

In [2]:
Map = geemap.Map(center=[10, -10], zoom=4)
Map

Map(center=[10, -10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## Add Earth Engine Python script

In [3]:


# WIO sym area
wio =  ee.Geometry.Polygon(
        [[[-1.1801562499999951, 22.240800146484577],
          [-1.1801562499999951, -40.201954377811155],
          [81.08546875, -40.201954377811155],
          [81.08546875, 22.240800146484577]]], None, False)
#**** End of imports. If edited, may not auto-convert in the playground. ****#
# This example demonstrates the use of the Landsat 8 Collection 2, Level 2
# QA_PIXEL band (CFMask) to mask unwanted pixels.

Map.addLayer(wio, {}, 'wio')

In [4]:
start_date = '2015-01-01',
end_date = '2022-12-31'

l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') 
                     .filterBounds(wio)
                     .filterDate(start_date, end_date) 
                     .maskL8sr)
def maskL8sr(l8):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = l8.select('QA_PIXEL').bitwiseAnd(parseInt('11111', 2)).eq(0)
    saturationMask = l8.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
    opticalBands = l8.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = l8.select('ST_B.*').multiply(0.00341802).add(149.0)

  # Replace the original bands with the scaled ones and apply the masks.
    return l8.addBands(opticalBands, None, True) \
                .addBands(thermalBands, None, True) \
                .updateMask(qaMask) \
                .updateMask(saturationMask)


# Map the function over one year of data.


l8_median = l8.median()

# Display the results.
Map.addLayer(l8_median, {'bands': ['SR_B4',  'SR_B3',  'SR_B2'], 'min': 0, 'max': 0.3})


Export.image.toDrive({
  'image' : composite,
  'description': 'L8_SR_wio_2015_2022',
  'maxPixels': 3784216672400,
  'region' : wio})

AttributeError: 'ImageCollection' object has no attribute 'maskL8sr'

## Display the interactive map

In [ ]:
Map